In [2]:
from keras.layers import Input, merge
from keras.models import Model,Sequential
from layers import VGGNormalize,ReflectionPadding2D,Denormalize,conv_bn_relu,res_conv,dconv_bn_relu
from loss import StyleReconstructionRegularizer,FeatureReconstructionRegularizer,TVRegularizer
import img_util

%matplotlib inline

%reload_ext autoreload

%autoreload 2


* Input 3 × 256 × 256
* Reflection Padding (40 × 40) 3 × 336 × 336
* 32 × 9 × 9 conv, stride 1 32 × 336 × 336
* 64 × 3 × 3 conv, stride 2 64 × 168 × 168
* 128 × 3 × 3 conv, stride 2 128 × 84 × 84
* Residual block, 128 filters 128 × 80 × 80
* Residual block, 128 filters 128 × 76 × 76
* Residual block, 128 filters 128 × 72 × 72
* Residual block, 128 filters 128 × 68 × 68
* Residual block, 128 filters 128 × 64 × 64
* 64 × 3 × 3 conv, stride 1/2 64 × 128 × 128
* 32 × 3 × 3 conv, stride 1/2 32 × 256 × 256
* 3 × 9 × 9 conv, stride 1 3 × 256 × 256

In [11]:
def image_transform_net():
    x = Input(shape=(256,256,3))
    a = ReflectionPadding2D(padding=(40,40),input_shape=(256,256,3))(x)
    a = conv_bn_relu(32, 9, 9, stride=(1,1))(a)
    a = conv_bn_relu(64, 9, 9, stride=(2,2))(a)
    a = conv_bn_relu(128, 3, 3, stride=(2,2))(a)
    for i in range(5):
        a = res_conv(128,3,3)(a)
    a = dconv_bn_relu(64,3,3)(a)
    a = dconv_bn_relu(32,3,3)(a)
    a = dconv_bn_relu(3,9,9,stride=(1,1))(a)
    # Scale output to range [0, 255] via custom Denormalize layer
    y = Denormalize(name='transform_output')(a)
    return  Model(input=x, output=y)

print image_transform_net().summary() 


____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_2 (InputLayer)             (None, 256, 256, 3)   0                                            
____________________________________________________________________________________________________
reflectionpadding2d_2 (Reflectio (None, 336, 336, 3)   0           input_2[0][0]                    
____________________________________________________________________________________________________
convolution2d_27 (Convolution2D) (None, 336, 336, 32)  7808        reflectionpadding2d_2[0][0]      
____________________________________________________________________________________________________
batchnormalization_14 (BatchNorm (None, 336, 336, 32)  128         convolution2d_27[0][0]           
___________________________________________________________________________________________

In [4]:
from keras.preprocessing.image import load_img, img_to_array
from scipy.misc import imsave
import numpy as np
from scipy.optimize import fmin_l_bfgs_b
import time
import argparse

from keras import backend as K


Image Util

In [5]:
from VGG16 import vgg16

img_width = 256
img_height = 256
style_weight=5.
content_weight=1.
tv_weight=1e-4
style_image_path = "images/style/starry_night.jpg"


def add_style_loss(vgg,style_image_path,vgg_layers,vgg_output_dict): 
    style_img = img_util.preprocess_image(style_image_path, img_width, img_height)
    print('Getting style features from VGG network.')

    style_layers = ['block1_conv2', 'block2_conv2', 'block3_conv3', 'block4_conv3']

    style_layer_outputs = []

    for layer in style_layers:
        style_layer_outputs.append(vgg_output_dict[layer])

    vgg_style_func = K.function([vgg.layers[-19].input], style_layer_outputs)    
        
    style_features = vgg_style_func([style_img])

    # Style Reconstruction Loss
    for i, layer_name in enumerate(style_layers):
        layer = vgg_layers[layer_name]
        
        feature_var = K.variable(value=style_features[i][0])
        style_loss = StyleReconstructionRegularizer(
                            style_feature_target=feature_var,
                            weight=style_weight)(layer)

        layer.add_loss(style_loss)

def add_content_loss(vgg_layers,vgg_output_dict):    
    # Feature Reconstruction Loss
    content_layer = 'block3_conv3'
    content_layer_output = vgg_output_dict[content_layer]

    layer = vgg_layers[content_layer]
    content_regularizer = FeatureReconstructionRegularizer(
                                       weight=content_weight)(layer)
    layer.add_loss(content_regularizer)    


def add_total_variation_loss(transform_output_layer):
    # Total Variation Regularization
    layer = transform_output_layer  # Output layer
    tv_regularizer = TVRegularizer(img_width=img_width, img_height=img_height,
                                   weight=tv_weight)(layer)
    layer.add_loss(tv_regularizer)    


def loss_net(x_in, trux_x_in):
    # Append the initial input to the FastNet input to the VGG inputs
    x = merge([x_in, trux_x_in], mode='concat', concat_axis=0)

    # Normalize the inputs via custom VGG Normalization layer
    x = VGGNormalize(name="vgg_normalize")(x)

    vgg = vgg16(include_top=False,input_tensor=x)

    vgg_output_dict = dict([(layer.name, layer.output) for layer in vgg.layers[-18:]])
    vgg_layers = dict([(layer.name, layer) for layer in vgg.layers[-18:]])
    
    add_style_loss(vgg,style_image_path , vgg_layers, vgg_output_dict)    
    add_content_loss(vgg_layers,vgg_output_dict)    
    
    # Freeze all VGG layers
    for layer in vgg.layers[-19:]:
        layer.trainable = False

    return vgg
    



model = image_transform_net()
#if tranning
add_total_variation_loss(model.layers[-1])
model = loss_net(model.output,model.input)

print model.summary()


VGG Model weights loaded.
Getting style features from VGG network.
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 256, 256, 3)   0                                            
____________________________________________________________________________________________________
reflectionpadding2d_1 (Reflectio (None, 336, 336, 3)   0           input_1[0][0]                    
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)  (None, 336, 336, 32)  7808        reflectionpadding2d_1[0][0]      
____________________________________________________________________________________________________
batchnormalization_1 (BatchNorma (None, 336, 336, 32)  128         convolution2d_1[0][0]            
________________________

In [10]:
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator

nb_epoch = 20
train_batchsize = 1 #4
train_image_path = "images/train/"

learning_rate = 0.1 #1e-3
optimizer = Adam(lr=learning_rate,beta_1=0.99)

dummy_loss = lambda y_true, y_pred : K.variable(0.0)

model.compile(optimizer,  dummy_loss)  # Dummy loss since we are learning from regularizes

datagen = ImageDataGenerator(rescale=1. / 255)

dummy_y = np.zeros((train_batchsize, img_height, img_width, 3)) # Dummy output, not used since we use regularizers to train


for i in range(nb_epoch):
    print("Epoch : %d" % (i + 1))

    for x in datagen.flow_from_directory(train_image_path, class_mode=None, batch_size=train_batchsize,
        target_size=(img_width, img_height), shuffle=False):    
        
        print("train image ")
        t1 = time.time()
        hist = model.fit(x, dummy_y, batch_size=train_batchsize, nb_epoch=2, verbose=0)
        loss = hist.history['loss'][0]
      
        print("done  %d %d ", loss,(time.time() -t1))
    

Epoch : 1
Found 4 images belonging to 1 classes.
train image 
('done  %d %d ', 4.2566528482829641e+23, 54.88097095489502)
train image 
('done  %d %d ', 4.2566528482829641e+23, 39.78511881828308)
train image 


KeyboardInterrupt: 

In [122]:
base_image_path = 
# get tensor representations of our images
base_image = K.variable(preprocess_image(base_image_path))
style_reference_image = K.variable(preprocess_image(style_reference_image_path))
combination_image = K.placeholder((1, img_nrows, img_ncols, 3))
    

# combine the 3 images into a single Keras tensor
input_tensor = K.concatenate([base_image,
                              style_reference_image,
                              combination_image], axis=0)

# build the VGG16 network with our 3 images as input
# the model will be loaded with pre-trained ImageNet weights
model = vgg16.VGG16(input_tensor=input_tensor,
                    weights='imagenet', include_top=False)


# combine these loss functions into a single scalar
loss = K.variable(0.)
layer_features = outputs_dict['block4_conv2']
base_image_features = layer_features[0, :, :, :]
combination_features = layer_features[2, :, :, :]
loss += content_weight * content_loss(base_image_features,
                                      combination_features)

feature_layers = ['block1_conv1', 'block2_conv1',
                  'block3_conv1', 'block4_conv1',
                  'block5_conv1']
for layer_name in feature_layers:
    layer_features = outputs_dict[layer_name]
    style_reference_features = layer_features[1, :, :, :]
    combination_features = layer_features[2, :, :, :]
    sl = style_loss(style_reference_features, combination_features)
    loss += (style_weight / len(feature_layers)) * sl
loss += total_variation_weight * total_variation_loss(combination_image)


def loss_net():
    x = Input(shape=(256,256,3))
    a = ReflectionPadding2D(padding=(40,40),input_shape=(256,256,3))(x)
    a = conv_bn_relu(32, 9, 9, stride=(1,1))(a)
    a = conv_bn_relu(64, 9, 9, stride=(2,2))(a)
    a = conv_bn_relu(128, 3, 3, stride=(2,2))(a)
    for i in range(5):
        a = res(128,3,3)(a)
    a = dconv_bn_relu(64,3,3)(a)
    a = dconv_bn_relu(32,3,3)(a)
    y = dconv_bn_relu(3,9,9,stride=(1,1))(a)
    return  Model(input=x, output=y)

NameError: name 'base_image_path' is not defined